In [18]:
import os, sys
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))
from src.models import TadGAN, AttentionTadGAN
from src.processor import AnomalyDataset
from src.configuration.constants import MODELS_DIRECTORY
from tqdm.notebook import tqdm
import numpy as np
import tensorflow as tf

In [19]:
def save_predictions(model, X_train, y_train, X_test, y_test):
    output_directory = os.path.join(MODELS_DIRECTORY, source, dataset, signal, model.model_name)
    y_hat, critic = model.predict(X_train, y_train)
    with open(os.path.join(output_directory, 'y_hat_train.npy'), 'wb') as f:
        np.save(f, y_hat)
    with open(os.path.join(output_directory, 'critic_train.npy'), 'wb') as f:
        np.save(f, critic)
              
    y_hat, critic = model.predict(X_test, y_test)
    with open(os.path.join(output_directory, 'y_hat_test.npy'), 'wb') as f:
        np.save(f, y_hat)
    with open(os.path.join(output_directory, 'critic_test.npy'), 'wb') as f:
        np.save(f, critic)

In [20]:
def train(source, dataset, signal, univariate=True):
    anomaly_dataset = AnomalyDataset.load(source, dataset, signal)
    X_train, y_train = anomaly_dataset.train.X, anomaly_dataset.train.y
    X_test, y_test = anomaly_dataset.test.X, anomaly_dataset.test.y
    
    if univariate:
        X_train = y_train
        X_test = y_test
    
    tadgan_model = TadGAN(
        input_shape=X_train[0].shape, 
        target_shape=y_train[0].shape,
    )
    if univariate:
        tadgan_model.model_name = 'univariate_tadgan'
    tadgan_model.fit(X_train, y_train, print_logs=False)
    tadgan_model.save(source, dataset, signal)
    save_predictions(tadgan_model, X_train, y_train, X_test, y_test)
    tf.keras.backend.clear_session()


    attention_tadgan_model = AttentionTadGAN(
        input_shape=X_train[0].shape, 
        target_shape=y_train[0].shape,
        num_heads=1,
    )
    if univariate:
        attention_tadgan_model.model_name = 'univariate_attention_tadgan'
    attention_tadgan_model.fit(X_train, y_train, print_logs=False)
    attention_tadgan_model.save(source, dataset, signal)
    save_predictions(attention_tadgan_model, X_train, y_train, X_test, y_test)
    tf.keras.backend.clear_session()

In [27]:
source = 'NASA'
dataset = 'SMAP'
signals = AnomalyDataset.get_signals(source, dataset)
signals = signals[50:]
signals

['T-3', 'E-13', 'P-2', 'R-1']

In [28]:
for signal in signals:
    try:
        train(source, dataset, signal)
        print(f'[{source}][{dataset}][{signal}][Done]')
    except Exception as e:
        print(f'[{source}][{dataset}][{signal}][{e}]')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 70/70 [06:39<00:00,  5.70s/it]


[NASA][SMAP][T-3][Done]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 70/70 [06:58<00:00,  5.97s/it]


[NASA][SMAP][E-13][Done]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 70/70 [07:42<00:00,  6.60s/it]


[NASA][SMAP][P-2][Done]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 70/70 [06:45<00:00,  5.79s/it]


[NASA][SMAP][R-1][Done]


In [ ]:
source = 'NASA'
dataset = 'MSL'
signals = AnomalyDataset.get_signals(source, dataset)

for signal in signals:
    try:
        train(source, dataset, signal)
        print(f'[{source}][{dataset}][{signal}][Done]')
    except Exception as e:
        print(f'[{source}][{dataset}][{signal}][{e}]')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 70/70 [06:32<00:00,  5.61s/it]


[NASA][MSL][M-5][Done]


 54%|██████████████████████████████████████████████████████████████████████████▉                                                               | 38/70 [04:10<03:14,  6.07s/it]